## Skin Cancer Classification

### Import required libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
# Read CSV file
df = pd.read_csv("data/hmnist_28_28_RGB.csv")
df.head()

,pixel0000,pixel0001,pixel0002,pixel0003,pixel0004,pixel0005,pixel0006,pixel0007,pixel0008,pixel0009,...,pixel2343,pixel2344,pixel2345,pixel2346,pixel2347,pixel2348,pixel2349,pixel2350,pixel2351,label
0,192,153,193,195,155,192,197,154,185,202,...,173,124,138,183,147,166,185,154,177,2
1,25,14,30,68,48,75,123,93,126,158,...,60,39,55,25,14,28,25,14,27,2
2,192,138,153,200,145,163,201,142,160,206,...,167,129,143,159,124,142,136,104,117,2
3,38,19,30,95,59,72,143,103,119,171,...,44,26,36,25,12,17,25,12,15,2
4,158,113,139,194,144,174,215,162,191,225,...,209,166,185,172,135,149,109,78,92,2


In [27]:
# Seperate features and labels
X = df.drop(['label'], axis=1)
y = df["label"]

### Synthetic Sampling

In [12]:
# Use SMOTE + ENN to generate synthetic examples for minority classes
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=42)
X_resampled, y_resampled_df = smote_enn.fit_resample(X, y)

In [13]:
# Reshape features
X_resampled = np.array(X_resampled).reshape(-1, 28, 28, 3)
X_resampled.shape

(43473, 28, 28, 3)

In [16]:
# One-hot-encode labels
from tensorflow.keras.utils import to_categorical

y_resampled = np.array(y_resampled_df)
y_resampled = to_categorical(y_resampled)
y_resampled.shape

(43473, 7)

### Model Definition and Training

In [18]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train , y_test = train_test_split(X_resampled , y_resampled , test_size = 0.2 , random_state = 42)

In [19]:
# Import libraries for data augmentation and creating the model
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, BatchNormalization, MaxPooling2D, Conv2D, Flatten

In [20]:
model = tf.keras.Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 3)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    
    Dense(128, activation='relu'),
    BatchNormalization(),
    
    Dense(64, activation='relu'),
    BatchNormalization(),
    
    Dense(64, activation='relu'),
    BatchNormalization(),
    
    Dense(7, activation='softmax')
])


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 64)     │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 7, 7, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 3, 3, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 3, 3, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 1, 1, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │             

 Total params: 634,311 (2.42 MB)

 Trainable params: 632,135 (2.41 MB)

 Non-trainable params: 2,176 (8.50 KB)

In [22]:
# Train the model for 20 epochs
history = model.fit(
    X_train, y_train, 
    epochs=20, 
    validation_data=(X_test, y_test)
)

Epoch 1/20
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step - accuracy: 0.9101 - loss: 0.2578 - val_accuracy: 0.5075 - val_loss: 1.7741
Epoch 2/20
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step - accuracy: 0.9303 - loss: 0.1996 - val_accuracy: 0.7090 - val_loss: 0.9359
Epoch 3/20
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step - accuracy: 0.9507 - loss: 0.1443 - val_accuracy: 0.8638 - val_loss: 0.3870
Epoch 4/20
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 40s 37ms/step - accuracy: 0.9619 - loss: 0.1161 - val_accuracy: 0.8615 - val_loss: 0.4327
Epoch 5/20
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 40s 37ms/step - accuracy: 0.9692 - loss: 0.0885 - val_accuracy: 0.8043 - val_loss: 0.5930
Epoch 6/20
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step - accuracy: 0.9747 - loss: 0.0794 - val_accuracy: 0.8928 - val_loss: 0.3126
Epoch 7/20
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step - accuracy: 0.9757 - loss: 0.0713 - val_accuracy: 0.8113 - val_loss: 0.5800
Epoch 8/20
1087/1087 ━━━━━━━━━━━━━━━━━━━━ 42s 38ms/step - accuracy: 0.9793 -

### Model Evaluation (Regular and F1)

In [23]:
test_score = model.evaluate(X_test, y_test, verbose=1)
print("Test Accuracy: ", test_score[1])

272/272 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.9842 - loss: 0.0566
Test Accuracy:  0.9833237528800964


In [24]:
from sklearn.metrics import f1_score

# Get predictions
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Calculate F1 score
score = f1_score(y_test, y_pred, average='weighted')
print("F1 Score:", score)

272/272 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
F1 Score: 0.9829880156561995


### Save Final Model

In [25]:
model.save(f"./saved_model/model_acc_{round(score, 3)}.h5", save_format='h5')